In [ ]:
#The following code will process the collected tweets and extract @Mentioned, Tweet Created User, and in reply to screen name

In [3]:
import json
import time
fls = ['rawData\condolence.json','rawData\RestInPeace.json','rawData\R_I_P.json','rawData\RIP.json']
sn = []
peopleLostSomeOneIn_reply_to_screen_name = set()
peopleLostSomeOneScreen_name = set()
snSet = set()
sn_Mentioned = []
for f in fls:
    print f
    tweets_data_path = f #'condolence.json'
    tweets_data = []
    RT_tweets = []
    tweets_file = open(tweets_data_path, "r")
    for line in tweets_file:
        try:
            tweet = json.loads(line)        
            tweets_data.append(tweet)
        except:
            continue
    print "Total Tweet Count : "+str(len(tweets_data) )

    for status in tweets_data:
        for user_mention in status['entities']['user_mentions']:
            screenName = user_mention['screen_name']
            dateMentioned = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(status['created_at'],'%a %b %d %H:%M:%S +0000 %Y'))
            if screenName in snSet:
                continue
            snSet.add(screenName)
            sn_Mentioned.append({'screenName': screenName, 'year' : dateMentioned})
        peopleLostSomeOneIn_reply_to_screen_name.add(status['in_reply_to_screen_name'])#In reply to there tweet
        peopleLostSomeOneScreen_name.add(status['user']['screen_name'])#Created The Tweet
    print "Total people we think they are dead: "+str(len(snSet))
with open('susbectedDead.json', 'w') as outfile:
    json.dump(sn_Mentioned, outfile)
outfile.close()
with open('lostSomeOne.txt', 'w') as thefile:
    for item in peopleLostSomeOneScreen_name.union(peopleLostSomeOneIn_reply_to_screen_name):
          thefile.write("%s\n" % item)
thefile.close()            

rawData\condolence.json
Total Tweet Count : 90010
Total people we think they are dead: 212
rawData\RestInPeace.json
Total Tweet Count : 90000
Total people we think they are dead: 405
rawData\R_I_P.json
Total Tweet Count : 90000
Total people we think they are dead: 666
rawData\RIP.json
Total Tweet Count : 90002
Total people we think they are dead: 1049


In [ ]:
#The following code will read the suspected list and lostsomeone list, and then check if the suspected account has any activity
#after mentioned on death related tweet to confirm more its suspected to be dead, else he will be added to the lost some list

In [1]:
import sys
import jsonpickle
import os
from tweepy import OAuthHandler
import tweepy
import time
import io
import json
from datetime import datetime

with open('susbectedDead.json') as data_file:    
    data = json.load(data_file)
    
with open('lostSomeOne.txt') as f:
    lines = f.read().splitlines()
    
print "Screen names mentioned on Death tweet that might be replied to: " + str(len(data))
print "Screen names of people how might lost some one (people created tweet/retweet or in_reply_to_user): " + str(len(lines))

i=0
while i < len(data):
    tmp = data[i]['screenName']
    if tmp in lines:
        data.pop(i)
    else:
        i += 1
print "Screen names of mentioned on Death tweets and not created the tweet or in_reply_to_user: " + str(len(data))


def write_to_json(new_tweets, fName):
    with open(fName, 'a') as datafile:
        for tweet in new_tweets:
            datafile.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')

def oauth_me():    
    CONSUMER_KEY = 'Y8PpyLf5UW0LSCdwTXHXI8Ypn'
    CONSUMER_SECRET ='BppAOhd8pLJxchOufdS3TTcv3F4CZptnUUHLtW65vsjv7oaJdl'
    OAUTH_TOKEN = '276846978-P2rWU1Dqthv3IRIP3sspbVCyhdmVrVrYqYVuqE1R'
    OAUTH_TOKEN_SECRET = 'ToLp2UEfMDIa9U8Z96g7wwjgS0bno1TxZhE8bpuitCO99'

    
    #OAuth with tweepy wrappers.
    auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
    
    # Construct the API instance
    api = tweepy.API(auth) # wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    print api
    return api

def processTweet(new_tweets,user,susbectedDeadNotActiveInfo):
    for status in new_tweets:
        #print data[j]['screenName']
        dateMentioned = datetime.strptime(data[j]['year'], '%Y-%m-%d %H:%M:%S')
        #print str(status.created_at) + "  VS  " + data[j]['year']
        if status.created_at > dateMentioned:
            print "He is not Dead: " + data[j]['screenName']
            linesSet.add(data[j]['screenName'])
            data[j]['screenName'] = '0'
        else:
            susbectedDeadNotActiveInfo.append({'screenName': data[j]['screenName'], 'LastAction' : str(status.created_at), 'userLocation' : user.location})
            
api = oauth_me()
print api
j = 0
linesSet = set(lines)
susbectedDeadNotActiveInfo = []
protectedSusbectProfile = set()
while j < len(data):
    try:
        user = api.get_user(data[j]['screenName'])
    except tweepy.TweepError as a:
        if 'Rate limit exceeded' in a.message[0]['message']:
            print "ERROR - " + str(a)
            print "Waiting 15 min"
            time.sleep(15*60)
        else:
            print "j = " + str(j) + " Couldn't Access ---------  " + data[j]['screenName']
        j += 1
        continue
    print data[j]['screenName']
    print str(user.protected)
    if user.protected:
        protectedSusbectProfile.add(data[j]['screenName'])
        j += 1  
        continue
    new_tweets = api.user_timeline(screen_name = data[j]['screenName'], count = 1, include_rts = True)
    try:
        processTweet(new_tweets, user, susbectedDeadNotActiveInfo)
        j += 1    
    except tweepy.TweepError as e:
        print "ERROR - " + str(e)
        time.sleep(15*60)
        processTweet(new_tweets, user, susbectedDeadNotActiveInfo)
        j += 1
        


Screen names mentioned on Death tweet that might be replied to: 1049
Screen names of people how might lost some one (people created tweet/retweet or in_reply_to_user): 2524
Screen names of mentioned on Death tweets and not created the tweet or in_reply_to_user: 743
olapaula20
False
He is not Dead: olapaula20
SkyNews
False
He is not Dead: SkyNews
Dev_Fadnavis
False
He is not Dead: Dev_Fadnavis
VanessaHudgens
False
He is not Dead: VanessaHudgens
Sunlight_Owl
False
He is not Dead: Sunlight_Owl
ericclarkeUTV
False
He is not Dead: ericclarkeUTV
Ryanschools
False
RyanIntlGrp
False
naqvi1966
False
He is not Dead: naqvi1966
BretHart
False
He is not Dead: BretHart
KalrajMishra
False
He is not Dead: KalrajMishra
thekiranbedi
False
He is not Dead: thekiranbedi
GoEnduro
False
He is not Dead: GoEnduro
HopeLoisGil
False
He is not Dead: HopeLoisGil
leniegammad
False
He is not Dead: leniegammad
AngPoetNyo
False
He is not Dead: AngPoetNyo
rodriguezruby
False
He is not Dead: rodriguezruby
allanklownz
Fa

C:\Users\IBM_ADMIN\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:74: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6


In [ ]:
#Printing the results after final processing

In [15]:
print "Number of people remained on suspected dead List : " + str(len(susbectedDeadNotActiveInfo))
print "Number of susbected dead has protect profiles : " + str(len(protectedSusbectProfile))
print "Number of people we think they lost someone : " + str(len(linesSet))
for x in susbectedDeadNotActiveInfo:
    print x['screenName'] + '\t' +  x['LastAction'] + '\t' + x['userLocation']
    #print x['screenName'] +'\t' , x['LastAction']+ '\t' +, x['userLocation']

Number of people remained on suspected dead List : 35
Number of susbected dead has protect profiles : 14
Number of people we think they lost someone : 3207
Ryanschools	2014-12-05 12:19:56	India and UAE
RyanIntlGrp	2016-01-07 09:26:03	India | UAE
ImranDhamrahPPP	2016-01-30 08:42:39	Sindh, Pakistan
GNev2	2016-01-09 17:06:30	
eiyooyie	2016-01-29 16:09:10	Olongapo City, Philippines
PyongyangPimp	2016-02-02 05:16:09	
hottunaband	2016-01-30 19:22:28	On the road...
MamaGH	2016-02-02 20:23:38	earth!
Sergioo_NikeAir	2013-01-05 06:26:03	Atlanta  ✔
dfeingoldphoto	2016-02-03 22:31:31	NYC
quickmixx	2016-01-30 17:36:54	JAMAICA
lawarehouselive	2016-01-07 04:05:19	Houston 
EarthWindFire	2016-02-04 22:57:33	
MarthaPlimpton	2016-02-04 23:02:02	New York
UWC_IO	2015-12-17 11:26:09	Worldwide
SUGIZOofficial	2016-01-31 06:23:47	Tokyo
ChinxMusic	2016-01-19 00:53:43	Queens, NY
krisndrob	2014-06-20 18:41:23	
itskekeloves__	2015-12-19 23:00:40	South CLT 
SpeakMemoryOk	2016-02-02 23:53:32	Oklahoma City, OK
Aidan2